In [54]:
import pandas as pd
import unicodedata
import re
import random, io
from MeCab import Tagger
import numpy as np
random.seed(1)
np.random.seed(1)

In [76]:
from sklearn.preprocessing import OneHotEncoder
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras import initializers, constraints, layers, callbacks
from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Input, BatchNormalization, LSTM, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, Add
from keras.layers import GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.utils import plot_model, to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

In [10]:
def read_excel(excel_file):
    sentences_sum = []
    sentences_tax = []
    sentences_total = []

    df = pd.read_excel(excel_file, sheet_name=0, keep_default_na=False)
    for row in range(0, df.shape[0]):

        # read and normalize unicode
        content_of_sum =  unicodedata.normalize('NFKC', df.loc[row, "sum detail of account "].lower())
        content_of_tax = unicodedata.normalize('NFKC', df.loc[row, "tax"].lower())
        content_of_total = unicodedata.normalize('NFKC', df.loc[row, "total amount "].lower())

        # replace space, ()〈〉 and split string
        sentences_sum.extend(re.sub('[ ]', '', content_of_sum).split('\n'))
        sentences_tax.extend(re.sub('[ ]', '', content_of_tax).split('\n'))
        sentences_total.extend(re.sub('[ ]', '', content_of_total).split('\n'))

    # Remove null
    sentences_sum = [phrase for phrase in sentences_sum if phrase]
    sentences_tax = [phrase for phrase in sentences_tax if phrase]
    sentences_total = [phrase for phrase in sentences_total if phrase]

    # get unique
    sentences_sum = list(set(sentences_sum))
    sentences_tax = list(set(sentences_tax))
    sentences_total = list(set(sentences_total))

    data = []

    data.extend([phrase, '1'] for phrase in sentences_sum)
    data.extend([phrase, '2'] for phrase in sentences_tax)
    data.extend([phrase, '3'] for phrase in sentences_total)
    return np.array(data)

In [11]:
class Tokenizer(Tagger):
    word_index = {}
    max_len = 0

    def __init__(self):
        Tagger.__init__(self)

    def tokenize(self, sequence):
        '''Tokenize a string using mecab'''
        sequence = sequence.replace(' ', '')
        sequence = ''.join([unicodedata.normalize('NFKC', char) for char in sequence])
        sequence = self.parse(sequence).splitlines()
        sequence = [line.split('\t')[0] for line in sequence]
        sequence = [token for token in sequence if token != 'EOS']
        sequence = [list(token) if token.isnumeric() else [token] for token in sequence]
        sequence = [token for sublist in sequence for token in sublist]
        return sequence

    def fit_on_texts(self, sentences):
        all_tokens = []
        for sentence in sentences:
            tokens = self.tokenize(sentence)
            all_tokens.extend(tokens)
            if len(tokens) > self.max_len:
                self.max_len = len(tokens)
        unique_tokens = np.unique(all_tokens)

        for i, token in enumerate(unique_tokens):
            self.word_index[token] = i + 1

    def texts_to_sequences(self, sentences):
        all_tokens = []
        for sentence in sentences:
            tokens = self.tokenize(sentence)
            tokenized = []
            for token in tokens:
                tokenized.append(self.word_index[token])
            all_tokens.append(tokenized)
        return all_tokens

In [20]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
    return data

In [39]:
def build_embedding_matrix(embedding_index, word_index, max_features, embed_size=300):
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [14]:
def create_data_for_other_class(list_words, max_len, number_sample, class_name):
    data = []
    list_words = np.array(list_words)
    for i in range(0, number_sample):
        random_size = random.randint(1, max_len)
        indices = np.random.randint(len(list_words), size=random_size)
        tokens = list_words[indices]
        sentences = ''.join(tokens)
        data.append([sentences, class_name])
    return np.array(data)

In [15]:
excel_file = '/Users/anh/Downloads/listkey.xlsx'
fasttext_file = '/Users/anh/Downloads/cc.ja.300.vec'

**Read data and preprocess**

In [16]:
excel_data = read_excel(excel_file)

**Load fasttext vector**

In [21]:
vector = load_vectors(fasttext_file)
list_words = list(vector.keys())

**Create data for class 4**

In [23]:
max_len = 20
num_sample = 50
data_other_class = create_data_for_other_class(list_words, max_len, num_sample, '4')

**Combine data**

In [30]:
data = np.concatenate((excel_data, data_other_class), axis=0)
X_data = data[:, :-1]
y = data[:, -1:].flatten()

**Tokenize**

In [38]:
sentences = X_data.flatten()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
tokenized = tokenizer.texts_to_sequences(sentences)
X_train = pad_sequences(tokenized, maxlen=max_len)

**Create embedding matrix**

In [40]:
embed_size = 300
word_index = tokenizer.word_index
max_features = len(word_index)
embedding_matrix = build_embedding_matrix(vector, word_index, max_features, embed_size)

**Convert y to one hot vector**

In [49]:
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.reshape(-1, 1))

**Build model 1**

In [87]:
# define callbacks
file_path = "Model1.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,
                              save_best_only=True, mode="min")
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=3)
callbacks_list = [check_point, early_stop]

# train
model = Sequential()
model.add(Embedding(max_features+1, embed_size,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(4, activation='sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

# model training
history = model.fit(X_train, y_ohe, batch_size=32, epochs=100, callbacks=callbacks_list,
                    validation_split=0.1, shuffle=True, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 20, 300)           153000    
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 20, 64)            57664     
_________________________________________________________________
global_max_pooling1d_24 (Glo (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 260       
Total params: 210,924
Trainable params: 57,924
Non-trainable params: 153,000
_________________________________________________________________
Train on 185 samples, validate on 21 samples
Epoch 1/100
185/185 [==============================] - 2s - loss: 0.6342 - acc: 0.6797 - val_loss: 0.6388 - val_acc: 0.7500
Epoch 2/100
185/185 [==============================] - 0s - loss: 0.4487 - acc: 0.8581 - val_l

185/185 [==============================] - 0s - loss: 0.0940 - acc: 0.9797 - val_loss: 0.2001 - val_acc: 1.0000
Epoch 27/100
185/185 [==============================] - 0s - loss: 0.0915 - acc: 0.9797 - val_loss: 0.1950 - val_acc: 1.0000
Epoch 28/100
185/185 [==============================] - 0s - loss: 0.0894 - acc: 0.9797 - val_loss: 0.1903 - val_acc: 1.0000
Epoch 29/100
185/185 [==============================] - 0s - loss: 0.0865 - acc: 0.9811 - val_loss: 0.1860 - val_acc: 1.0000
Epoch 30/100
185/185 [==============================] - 0s - loss: 0.0855 - acc: 0.9784 - val_loss: 0.1815 - val_acc: 1.0000
Epoch 31/100
185/185 [==============================] - 0s - loss: 0.0827 - acc: 0.9784 - val_loss: 0.1774 - val_acc: 1.0000
Epoch 32/100
185/185 [==============================] - 0s - loss: 0.0819 - acc: 0.9784 - val_loss: 0.1732 - val_acc: 1.0000
Epoch 33/100
185/185 [==============================] - 0s - loss: 0.0795 - acc: 0.9797 - val_loss: 0.1691 - val_acc: 1.0000
Epoch 34/100


185/185 [==============================] - 0s - loss: 0.0537 - acc: 0.9811 - val_loss: 0.1065 - val_acc: 1.0000
Epoch 55/100
185/185 [==============================] - 0s - loss: 0.0536 - acc: 0.9811 - val_loss: 0.1046 - val_acc: 1.0000
Epoch 56/100
185/185 [==============================] - 0s - loss: 0.0527 - acc: 0.9811 - val_loss: 0.1025 - val_acc: 1.0000
Epoch 57/100
185/185 [==============================] - 0s - loss: 0.0521 - acc: 0.9811 - val_loss: 0.1005 - val_acc: 1.0000
Epoch 58/100
185/185 [==============================] - 0s - loss: 0.0512 - acc: 0.9784 - val_loss: 0.0986 - val_acc: 1.0000
Epoch 59/100
185/185 [==============================] - 0s - loss: 0.0507 - acc: 0.9811 - val_loss: 0.0968 - val_acc: 1.0000
Epoch 60/100
185/185 [==============================] - 0s - loss: 0.0503 - acc: 0.9811 - val_loss: 0.0949 - val_acc: 1.0000
Epoch 61/100
185/185 [==============================] - 0s - loss: 0.0498 - acc: 0.9811 - val_loss: 0.0931 - val_acc: 1.0000
Epoch 62/100


185/185 [==============================] - 0s - loss: 0.0411 - acc: 0.9784 - val_loss: 0.0649 - val_acc: 1.0000
Epoch 84/100
185/185 [==============================] - 0s - loss: 0.0401 - acc: 0.9811 - val_loss: 0.0640 - val_acc: 1.0000
Epoch 85/100
185/185 [==============================] - 0s - loss: 0.0416 - acc: 0.9797 - val_loss: 0.0631 - val_acc: 1.0000
Epoch 86/100
185/185 [==============================] - 0s - loss: 0.0397 - acc: 0.9811 - val_loss: 0.0621 - val_acc: 1.0000
Epoch 87/100
185/185 [==============================] - 0s - loss: 0.0393 - acc: 0.9811 - val_loss: 0.0611 - val_acc: 1.0000
Epoch 88/100
185/185 [==============================] - 0s - loss: 0.0405 - acc: 0.9784 - val_loss: 0.0604 - val_acc: 1.0000
Epoch 89/100
185/185 [==============================] - 0s - loss: 0.0391 - acc: 0.9824 - val_loss: 0.0595 - val_acc: 1.0000
Epoch 90/100
185/185 [==============================] - 0s - loss: 0.0391 - acc: 0.9797 - val_loss: 0.0587 - val_acc: 1.0000
Epoch 91/100


**Build model 2**

In [65]:
# define callbacks
file_path = "Model2.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,
                              save_best_only=True, mode="min")
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=3)
callbacks_list = [check_point, early_stop]

# train
model = Sequential()
model.add(Embedding(max_features+1, embed_size,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Dense(4, activation='sigmoid'))  # multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

# model training
history = model.fit(X_train, y_ohe, batch_size=32, epochs=100, callbacks=callbacks_list,
                    validation_split=0.1, shuffle=True, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 20, 300)           153000    
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 20, 64)            57664     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 10, 64)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 10, 64)            12352     
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 64)                0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
__________

185/185 [==============================] - 0s - loss: 0.1043 - acc: 0.9716 - val_loss: 0.1465 - val_acc: 1.0000
Epoch 25/100
185/185 [==============================] - 0s - loss: 0.0913 - acc: 0.9716 - val_loss: 0.1348 - val_acc: 1.0000
Epoch 26/100
185/185 [==============================] - 0s - loss: 0.0854 - acc: 0.9770 - val_loss: 0.1242 - val_acc: 1.0000
Epoch 27/100
185/185 [==============================] - 0s - loss: 0.0844 - acc: 0.9811 - val_loss: 0.1120 - val_acc: 1.0000
Epoch 28/100
185/185 [==============================] - 0s - loss: 0.0901 - acc: 0.9689 - val_loss: 0.1076 - val_acc: 1.0000
Epoch 29/100
185/185 [==============================] - 0s - loss: 0.0828 - acc: 0.9730 - val_loss: 0.1020 - val_acc: 1.0000
Epoch 30/100
185/185 [==============================] - 0s - loss: 0.0998 - acc: 0.9649 - val_loss: 0.0989 - val_acc: 1.0000
Epoch 31/100
185/185 [==============================] - 0s - loss: 0.0907 - acc: 0.9743 - val_loss: 0.0934 - val_acc: 1.0000
Epoch 32/100


185/185 [==============================] - 0s - loss: 0.0626 - acc: 0.9838 - val_loss: 0.0360 - val_acc: 1.0000
Epoch 54/100
185/185 [==============================] - 0s - loss: 0.0550 - acc: 0.9797 - val_loss: 0.0346 - val_acc: 1.0000
Epoch 55/100
185/185 [==============================] - 0s - loss: 0.0540 - acc: 0.9770 - val_loss: 0.0348 - val_acc: 1.0000
Epoch 56/100
185/185 [==============================] - 0s - loss: 0.0454 - acc: 0.9838 - val_loss: 0.0366 - val_acc: 1.0000
Epoch 57/100
185/185 [==============================] - 0s - loss: 0.0571 - acc: 0.9757 - val_loss: 0.0393 - val_acc: 1.0000
Epoch 58/100
185/185 [==============================] - 0s - loss: 0.0553 - acc: 0.9770 - val_loss: 0.0422 - val_acc: 1.0000


**Build model 3**

In [92]:
file_path = "Model3.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)

def build_model1(lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features+1, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(spatial_dr)(x)

    x_gru = Bidirectional(GRU(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool3_gru = GlobalAveragePooling1D()(x3)
    max_pool3_gru = GlobalMaxPooling1D()(x3)
    
    x_lstm = Bidirectional(LSTM(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool3_lstm = GlobalAveragePooling1D()(x3)
    max_pool3_lstm = GlobalMaxPooling1D()(x3)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                    avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
    x = Dense(4, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.summary()
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, y_ohe, batch_size = 32, epochs = 100, validation_split=0.1, 
                        verbose = 1, callbacks = [check_point, early_stop])

In [93]:
model1 = build_model1(lr = 1e-3, lr_d = 0, units = 64, spatial_dr = 0.5, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 20)            0                                            
____________________________________________________________________________________________________
embedding_20 (Embedding)         (None, 20, 300)       153000      input_4[0][0]                    
____________________________________________________________________________________________________
spatial_dropout1d_4 (SpatialDrop (None, 20, 300)       0           embedding_20[0][0]               
____________________________________________________________________________________________________
bidirectional_5 (Bidirectional)  (None, 20, 128)       140160      spatial_dropout1d_4[0][0]        
___________________________________________________________________________________________

185/185 [==============================] - 0s - loss: 0.2465 - acc: 0.9176 - val_loss: 0.5571 - val_acc: 0.8333
Epoch 8/100
185/185 [==============================] - 0s - loss: 0.2152 - acc: 0.9176 - val_loss: 0.5687 - val_acc: 0.8095
Epoch 9/100
185/185 [==============================] - 0s - loss: 0.2103 - acc: 0.9270 - val_loss: 0.5658 - val_acc: 0.8095
Epoch 10/100
185/185 [==============================] - 0s - loss: 0.1908 - acc: 0.9365 - val_loss: 0.5408 - val_acc: 0.8214
Epoch 11/100
185/185 [==============================] - 0s - loss: 0.1803 - acc: 0.9338 - val_loss: 0.5190 - val_acc: 0.8333
Epoch 12/100
185/185 [==============================] - 0s - loss: 0.1590 - acc: 0.9392 - val_loss: 0.5149 - val_acc: 0.8214
Epoch 13/100
185/185 [==============================] - 0s - loss: 0.1492 - acc: 0.9378 - val_loss: 0.5440 - val_acc: 0.8095
Epoch 14/100
185/185 [==============================] - 0s - loss: 0.1776 - acc: 0.9284 - val_loss: 0.5634 - val_acc: 0.7857
Epoch 15/100
18

In [100]:
model1_file_path = 'Model1.hdf5'
model1 = load_model(model1_file_path)
score1 = model1.evaluate(X_train, y_ohe, verbose = 1)
print(score1)

 32/206 [===>..........................] - ETA: 8s[0.037539691130802466, 0.98300970873786409]


In [102]:
model2_file_path = 'Model2.hdf5'
model2 = load_model(model2_file_path)
score = model2.evaluate(X_train, y_ohe, verbose = 1)
print(score2)

 32/206 [===>..........................] - ETA: 9s[0.038978623533711849, 0.98300970873786409]


In [101]:
model3_file_path = 'Model3.hdf5'
model3 = load_model(model3_file_path)
score3 = model3.evaluate(X_train, y_ohe, verbose = 1)
print(score3)


192/206 [==========================>...] - ETA: 0s[0.1797583340441139, 0.94417475843892518]
